In [1]:
from datetime import datetime
import time
import gspread
import pandas as pd
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
client = gspread.authorize(creds)

In [2]:
def days_sum(day, change):
    date = day + change
    return date

In [3]:
entry_point = time.mktime((2004, 2, 6, 0, 0, 0, 0, 0, 0)) # reference for math - first Friday of February 2004

Tuesday = time.mktime((2004, 2, 10, 0, 0, 0, 0, 0, 0)) # reference for math with four days of difference

first_draw = time.mktime((2004, 2, 13, 0, 0, 0, 0, 0, 0)) # this is the first draws of Euro Millions

change_draw = time.mktime((2011, 5, 6, 0, 0, 0, 0, 0, 0)) # this is the Last draw of the old rule: one draw per week

next_week = first_draw - entry_point
next_Tuesday = Tuesday - entry_point
Tuesday_Friday = first_draw - Tuesday

In [4]:
dates = []
dates1 = []

for i in range(378):
    entry_point = days_sum(entry_point, next_week)
    dates.append(entry_point)

for i in range(379, 1598):
    if i % 2:
        change_draw = days_sum(change_draw, next_Tuesday)
    else:
        change_draw = days_sum(change_draw, Tuesday_Friday)
    dates1.append(change_draw)

df = pd.DataFrame(list(map(datetime.fromtimestamp, dates)))
df1 = pd.DataFrame(list(map(datetime.fromtimestamp, dates1)))

frames = [df, df1]
result = pd.concat(frames, ignore_index=True)
result = result.rename(columns={0:'Dates'})

In [5]:
sh = client.open_by_key('1nJmpNVlyePB09EnMFET_CAsLccu6hs-IP3fsRBG0UXQ')
worksheet = sh.get_worksheet(2)
data = worksheet.get_all_values()
draws_df = pd.DataFrame(data)
draws_df = draws_df[[0, 1, 2, 3, 4, 5, 6, 7]]
draws_df.columns = ['Sorteos', 'Nro1', 'Nro2', 'Nro3', 'Nro4', 'Nro5', 'Star_1', 'Star_2']
indexes_to_drop = [0, 1, 2, 3, 1601, 1602, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617]
indexes_to_keep = set(range(draws_df.shape[0])) - set(indexes_to_drop)
draws_df = draws_df.take(list(indexes_to_keep))
draws_df.reset_index(drop=True, inplace=True)
df_clean = {
    'Sorteos': "string",
    'Nro1': "int64",
    'Nro2': "int64",
    'Nro3': "int64",
    'Nro4': "int64",
    'Nro5': "int64",
    'Star_1': "int64",
    'Star_2': "int64"
}
draws_df = draws_df.astype(df_clean)
draws_df = pd.concat([result, draws_df], axis=1, join='inner')
draws_df

,Dates,Sorteos,Nro1,Nro2,Nro3,Nro4,Nro5,Star_1,Star_2
0,2004-02-13,1,16,29,32,36,41,7,9
1,2004-02-20,2,7,13,39,47,50,2,5
2,2004-02-27,3,14,18,19,31,37,4,5
3,2004-03-05,4,4,7,33,37,39,1,5
4,2004-03-12,5,15,24,28,44,47,4,5
...,...,...,...,...,...,...,...,...,...
1592,2022-12-27,1593,19,31,33,34,43,3,5
1593,2022-12-30,1594,17,34,42,47,48,1,9
1594,2023-01-03,1595,20,21,29,45,46,3,10
1595,2023-01-06,1596,16,31,35,44,45,4,12


In [6]:
draws_df.dtypes

Dates      datetime64[ns]
Sorteos            string
Nro1                int64
Nro2                int64
Nro3                int64
Nro4                int64
Nro5                int64
Star_1              int64
Star_2              int64
dtype: object

In [7]:
draws_df.to_csv('base_de_datos.csv', index=False)

In [8]:
# row_189 = draws_df[draws_df["Sorteos"] == "378"]
# print(row_189)